In [1]:
!pip install pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 742.7/742.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 26.0
    Uninstalling packaging-26.0:
      Successfully uninstalled packaging-26.0


In [1]:
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

# Initialize Pinecone
pc = Pinecone(api_key="Your_API_KEY")
#Copy Key from your pinecone account

# Create index (only once)
index_name = "demo-index"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=3977,  # depends on embedding model
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

index = pc.Index(index_name)

In [2]:
!pip install pypdf

import pandas as pd
import numpy as np
from pypdf import PdfReader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.5/331.5 kB 6.1 MB/s eta 0:00:00


In [3]:
# Define the path to your PDF file
path = "/content/sample_data/Foundations_of_Machine_Learning.pdf"

#extract text from pdf
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() or "" # Use or "" to handle pages with no extractable text
    return text

docs = extract_text_from_pdf(path)

In [4]:
# Simple chunking
chunk_size = 200
chunks = [docs[i:i+chunk_size]
          for i in range(0, len(docs), chunk_size)]

print("Total chunks:", len(chunks))
print(chunks)

Total chunks: 3977
['Foundations of Machine LearningAdaptive Computation and Machine Learning\nThomas Dietterich, Editor\nChristopher Bishop, David Heckerman, Michael Jordan, and Michael Kearns,\nAssociate Editors\nA complete', ' list of books published in The Adaptive Computations and Machine\nLearning series appears at the back of this book.Foundations of Machine Learning\nMehryar Mohri, Afshin Rostamizadeh, and Ameet Talwalk', 'ar\nThe MIT Press\nCambridge, Massachusetts\nLondon, Englandc⃝ 2012 Massachusetts Institute of Technology\nAll rights reserved. No part of this book may be reproduced in any form by any\nelectronic or mech', 'anical means (including photocopying, recording, or information\nstorage and retrieval) without permission in writing from the publisher.\nMIT Press books may be purchased at special quantity discounts ', 'for business or\nsales promotional use. For information, please email special\nsales@mitpress.mit.edu\nor write to Special Sales Department, The MIT P

In [5]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks)

len(embeddings)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

3977

In [6]:
vectors = []

for i, doc in enumerate(chunks):
    vectors.append({
        "id": f"doc_{i}",
        "values": embeddings[i].tolist(),
        "metadata": {"text": doc}
    })

#If you store full PDFs / long text inside metadata, payload size explodes.
#If still error → reduce batch_size to 50 or 20.

def batch_upsert(index, vectors, batch_size=200):
    for i in range(0, len(vectors), batch_size):
        batch = vectors[i:i+batch_size]
        index.upsert(vectors=batch)

batch_upsert(index, vectors, batch_size=200)

In [7]:
query = "What is a machine learning?"
query_embedding = model.encode([query])[0]

results = index.query(
    vector=query_embedding.tolist(),
    top_k=2,
    include_metadata=True
)

for match in results["matches"]:
    print(match["metadata"]["text"], "Score:", match["score"])

 to
the success of the predictions made by the learner.
Machine learning consists of designing eﬃcient and accurate prediction algo-
rithms. As in other areas of computer science, some critical measur Score: 0.709820807
mplementation or their adaptation to other learning scenarios.
Machine learning is a relatively recent ﬁeld and yet probably one of the most
active ones in computer science. Given the wide accessibili Score: 0.701968253
